# Header

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits.mplot3d import Axes3D
import scipy as s
%matplotlib inline
#%matplotlib notebook

In [2]:
# Check directory
os.chdir("D:\Raphael\Dropbox\Mcgill\JupyterNotebook\PHYS512")
os.getcwd()

'D:\\Raphael\\Dropbox\\Mcgill\\JupyterNotebook\\PHYS512'

# Functions

In [3]:
# Generate random inital condition
def init_cond(n):
    x=np.random.rand(n,3)
    v=np.random.rand(n,3)*0
    m=np.ones(n)/n
    return x,v,m

In [52]:
# Define softened potential, given a density matrix shape
def soft_pot(rho,edge,a):
    pot =rho*0 #initialise potential matrix
    nx = len(rho[0]) # number of bins in x
    ny = len(rho[1])
    nz = len(rho[2])
    dx = edge[0][1]-edge[0][0] # size of bin in x
    dy = edge[1][1]-edge[1][0]
    dz = edge[2][1]-edge[2][0]
    for i in range(nx):
        for j in range(ny):
            for k in range(nz): # looking at some bin i,j,k
                x = i+dx-0.5 # position of the center of the bin in x, given ptcl at x=0.5 (i.e at the center, since x=[0,1) )
                y = j+dy-0.5
                z = k+dz-0.5
                r = np.sqrt(x**2+y**2+z**2) # distance btw center of bin and and particle
                if r**3<=a:
                    pot[i,j,k] = r/a # potential for bins close to ptcl
                else:
                    pot[i,j,k] = 1/r**2 # potential for bins far from ptcl
    return pot

In [5]:
#Return a 3D plot of the given x
def plot_graph3D(x, title ="", scale_factor = 1,save=False, name = "fig",n = 0):
    #Create figure
    fig=plt.figure(figsize=(10,10))#Create 3D axes
    ax=Axes3D(fig) 
    ax.scatter(x[0], x[1], x[2], color="royalblue", marker=".",s=.02*scale_factor)
    ax.set_xlabel("x-coordinate",fontsize=14)
    ax.set_ylabel("y-coordinate",fontsize=14)
    ax.set_label("z-coordinate",fontsize=14)
    ax.set_title(title,fontsize=20)
    #ax.legend(loc="upper left",fontsize=14)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.zaxis.set_ticklabels([])
    if save:
        plt.savefig('Pictures_Final_Project/{}{}.png'.format(name,n), dpi=100)
    plt.close(fig)

# Classes

In [6]:
class Particles:
    def __init__(self,x,v,m):
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
        self.f=np.empty(x.shape)
        self.n=self.x.shape[0]
    def get_forces(self):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    r=np.sqrt(drsqr)
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]/(r*drsqr)
    def update(self,dt):
        self.get_forces()
        self.x=self.x+dt*self.v
        self.v=self.v+dt*self.f

# Code

In [56]:
# initialise randomly for some n ptcls in a box size^3
n = 100
size = 100
x,v,m = init_cond(n)

# Create density matrix rho
rho,edge = np.histogramdd(x,bins=size)

# Create softened potential matrix s_pot
s_pot = soft_pot(rho,edge,0.01)

# Create total potential matrix by convolving the density matrix with the softened potential
phi = s.ifft(s.fft(s_pot) * s.fft(rho))

In [30]:
#x[ptcl,xyz]
#x_t[xyz,ptcl]

In [ ]:
n=2
x=np.random.rand(n,3)
v=np.random.rand(n,3)*0
m=np.ones(n)
parts=Particles(x,v,m)
dt=0.1
for i in range(50):
    plot_graph3D(np.transpose(x),scale_factor=10000,save=True,n=i)
    parts.update(dt)

In [136]:
# define a cube to define the softened potential
size = 101
x = np.zeros(shape=(size**3,3))
x_i = 0
for i in range(size):
    for j in range(size):
        for k in range(size):
            x[x_i] = np.array([i-(size-1)/2,j-(size-1)/2,k-(size-1)/2])
            x_i = x_i + 1
G_laplacian = soft_pot(x,1)

In [124]:
rho = x_i*0

phi = s.ifft(s.fft(G_laplacian) * s.fft(rho))

ValueError: Invalid axis (-1) specified.